<a href="https://colab.research.google.com/github/smthomas1704/restoration-rag/blob/main/functional_trait_rag_with_jina_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download
Download the zipped literature file from this Google Drive location

In [ ]:
!git clone https://github.com/smthomas1704/restoration-rag.git

!pip install -r restoration-rag/requirements.txt
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4
!pip install gdown==v4.6.3
!pip install openai
!pip install langchain_experimental

# !gdown https://drive.google.com/file/d/10_inKhFuY5O8Sel88ZvlTqODsbbh4ula/view?usp=drive_link -O /content/functional_trait_literature_zipped.zip --fuzzy

# !unzip /content/functional_trait_literature_zipped.zip

# Chunk and store

In this portion, we'll chunk all the files into smaller paragraphs and use that for generating embeddings. We'll separately store the chunks so we can use it later, without having to re-download all the literature again.

### References/Notes related to chunking
1. https://openai.com/blog/new-and-improved-embedding-model
2. text-embedding-ada-002 is the best model for text embedding generation
3. https://www.pinecone.io/learn/chunking-strategies/
4. https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf#using-pypdf

### Possible strategies for chunking
1. LaTex: LaTeX is a document preparation system and markup language often used for academic papers and technical documents. By parsing the LaTeX commands and environments, you can create chunks that respect the logical organization of the content (e.g., sections, subsections, and equations), leading to more accurate and contextually relevant results.
2. Latex taxes a string as input, so we will need to read
3. Most of these academic papers are written in Latex and then converted to PDF. Our best bet would be to convert the PDF to Latex format first and then use the Latex based chunker to chunk things. This way paragraphs and related information will be together and contextualized.
4. On the other hand its not a guarantee that the document was first written in LaTex.

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from huggingface_hub import hf_hub_download
from langchain.docstore.document import Document
import pandas as pd
import json

REPO_ID = "collaborativeearth/functional_trait_papers"
FILENAME = "all_afr_carbon_large_chunks.jsonl"

# Download the chunks from Huggingface. We generated the chunks and uploaded it to Huggingface
file_name = hf_hub_download(repo_id=REPO_ID, filename=FILENAME, repo_type="dataset", local_dir="/content/")

print(file_name)
chunks=[]

with open(file_name, "r") as final:
  chunks = json.load(final)

prod_splits=[]

for chunk in chunks:
  try:
    prod_splits.append(Document(
      page_content=chunk['page_content'],
      metadata={
          "source": chunk['title'],
          "id": chunk['id']
      }
    ))
  except:
    print("Exception for chunk:")
    print(chunk)

print(prod_splits[0])
print(prod_splits[1])

all_afr_carbon_large_chunks.jsonl:   0%|          | 0.00/24.8M [00:00<?, ?B/s]

/content/all_afr_carbon_large_chunks.jsonl
page_content='\nSecondary forests comprise an increasing area of the tropics and play an important role in global carbon cycling. We compare above-ground biomass accumulation of both planted and naturally regenerating trees, as well as C in the top soil layer, in three restoration treatments replicated at 14, six to eight year old restoration sites in southern Costa Rica. Restoration strategies include: control (no planting), planting tree islands, and conventional, mixed-species tree plantations. We evaluate the importance of past landuse, soil nutrients, understory cover, and surrounding forest cover in explaining variation in aboveground biomass accumulation (ABA) rate across sites. Total ABA and planted tree ABA rate were highest in plantations, intermediate in islands, and lowest in control treatments, whereas ABA rate of naturally regenerating trees did not differ across treatments. Most ABA in plantations (89%) and islands (70%) was due

## Now we will generate embeddings using different models for comparison
First one is jina-embedding-l-en-v1

In [9]:
!mkdir "/content/vectorstore"
DB_JINA_EMBEDDING_PATH = 'vectorstore/db_jina-embedding-l-en-v1'

# Note this transformer version only supports context window of 512
# This is roughyl 384 words. Some of the sentences in our corpus is around
# 650 tokens. These will get truncated. So we should also try this out with
# the smaller embedding size.
embeddings = HuggingFaceEmbeddings(model_name='jinaai/jina-embedding-l-en-v1',
                                       model_kwargs={'device': 'cuda'})

db = FAISS.from_documents(prod_splits, embeddings)
serialized_bytes = db.serialize_to_bytes()
with open("/content/vectorstore/serialized_db.txt", "wb") as binary_file:
    # Write bytes to file
    binary_file.write(serialized_bytes)

db.save_local(DB_JINA_EMBEDDING_PATH)

mkdir: cannot create directory ‘/content/vectorstore’: File exists


modules.json:   0%|          | 0.00/253 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/58.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

In [21]:
query = "What are the key findings regarding the contribution of perennial plants and trees to the total aboveground carbon pool in different regions?"
docs = await db.asimilarity_search(query)
for doc in docs:
  print(doc)

page_content='Carbon pools of tree layers increased rapidly from 1994 to 2003 (ranging from 94.91 to 130.84 Tg). In Fig. 3, carbon storage of understory layer had an increasing trend from 1994 to 1996 (ranging from 34.88 to 37.79 Tg), followed by a slight decreasing trend from 1996 to 2003 (ranging from 37.79 to 36.79 Tg). Carbon storage of litterfall layer had a slightly increasing trend from 39.82 to 43.50 Tg over the 10-year period. Carbon storage of the understory vegetation layer was slightly less than that of the litterfall layer. Total carbon storage in understory vegetation and litterfall accounted for 38%-44% of the total forest carbon pools. This clearly shows that understory vegetation and litterfall should not be ignored when estimating total carbon pools in sub-tropical forest ecosystems.' metadata={'source': 'Impacts of a large-scale reforestation program on carbon storage dynamics in Guangdong, China', 'id': '217.12'}
page_content='Aim To analyse global patterns in soil 

In [23]:
from google.colab import userdata
from openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
import tiktoken
import os

OPENAI_API_KEY = userdata.get('COLABORATIVE_EARTH_KEY')
llm_model = "gpt-3.5-turbo"

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

llm = ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo")

memory = ConversationBufferMemory(
  memory_key='chat_history',
  return_messages=False
)

retriever = db.as_retriever(
    search_kwargs={"k": 5}
)

custom_template = """You are an AI assistant for assisted restoration papers.
You are given the following extracted parts of a long document.
=========
{context}
=========
Provide a conversational answer for the following question
Question: {question}.
If you don't know the answer, just say "Hmm, I'm not sure." Don't try to make up an answer.
Answer in Markdown:"""


conversation_chain_without_reference = ConversationalRetrievalChain.from_llm(
        llm=llm,
        # chain_type="stuff" will go through everything.
        # chain_type="refine",
        chain_type="stuff",
        retriever=retriever,
        # return_source_documents=True,
        verbose=True,
        memory=memory,
        # combine_docs_chain_kwargs={"question_prompt": custom_prompt, "refine_prompt": custom_prompt}
)

result = conversation_chain_without_reference({"question": query})
answer = result["answer"]

answer



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Carbon pools of tree layers increased rapidly from 1994 to 2003 (ranging from 94.91 to 130.84 Tg). In Fig. 3, carbon storage of understory layer had an increasing trend from 1994 to 1996 (ranging from 34.88 to 37.79 Tg), followed by a slight decreasing trend from 1996 to 2003 (ranging from 37.79 to 36.79 Tg). Carbon storage of litterfall layer had a slightly increasing trend from 39.82 to 43.50 Tg over the 10-year period. Carbon storage of the understory vegetation layer was slightly less than that of the litterfall layer. Total carbon storage in understory vegetation and litterfall accounted for 38%-44% of the total forest carbon pools. This clearly shows that understory vegetation and litterfall sh

'The key findings regarding the contribution of perennial plants and trees to the total aboveground carbon pool in different regions are as follows:\n\n1. In subtropical forest ecosystems, understory vegetation and litterfall play a significant role in contributing to total carbon pools, accounting for 38%-44% of the total forest carbon pools.\n2. Carbon storage levels in all vegetation layers, including trees, shrubs, and herbaceous plants, vary in different forest stands. For example, in CH, PM, and mixed CH/PM stands, the majority of carbon storage is attributed to trees and shrubs, with trees having the highest carbon storage in stems.\n3. Fast-growing tree species, such as EU and AC, in subtropical regions like southern China, have larger carbon storages compared to other plantation types. The understory vegetation also significantly contributes to plant carbon storage in subtropical plantation ecosystems.\n4. In mixed plantations, overstory tree carbon stocks were lower compared 

###TODO
1. Add a cross encoder after the retrieval stage to re-rank the results before feeding it to the API.
2. Alternately, or maybe along with this, we may also want to combine several chunks to send as context, depending on the context lenght.
3. Or perhaps we also chunk in small portions and combine results to send to OpenAI. This way answers can be formed based on chunks from different papers and different sections.
4. Update this RAG to cite sources from the context provided. Refence: https://blog.langchain.dev/langchain-chat/

https://towardsdatascience.com/4-ways-of-question-answering-in-langchain-188c6707cc5a